\begin{aligned}
-\Delta u & = f && \text {En una resistencia eléctrica},
\\
&\alpha u − \beta = 0 && \text{Con condición en la frontera tipo Robin}
\end{aligned}

In [1]:
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Circle, Rectangle

- Crear un objeto de geometría 2D utilizando CSG (Constructive Solid Geometry)
- Definir el primer círculo con centro en (4 + 4 * sqrt(2), 2), radio 6, material "mat1" y condición de borde "bc_circle1"
- Definir el segundo círculo con centro en (4 + 4 * sqrt(2), 2), radio 5, material "mat2" y condición de borde "bc_circle2"
- Definir el primer rectángulo con el punto mínimo en (0, 0) y el punto máximo en (4, 4), material "mat1" y condición de borde "bc_rect1"
- Definir el segundo rectángulo con el punto mínimo en (-0.5, 1) y el punto máximo en (5.5, 3), material "mat2" y condición de borde "bc_rect2"
- Crear el tercer dominio como la diferencia entre la unión del primer círculo con el primer rectángulo y la unión del segundo círculo con el segundo rectángulo
- Añadir el tercer dominio a la geometría
- Generar la malla con una máxima longitud de borde de 0.3
- Dibujar a malla


In [2]:
geo = CSG2d()
circle1 = Circle( center=(4 + 4 * np.sqrt(2),2), radius=6, mat="mat1", bc="bc_circle1" )
circle2 = Circle( center=(4 + 4 * np.sqrt(2),2), radius=5, mat="mat2", bc="bc_circle2" )
rect1 = Rectangle( pmin=(0,0), pmax=(4,4), mat="mat1", bc="bc_rect1" )
rect2 = Rectangle ( pmin=(-0.5,1), pmax=(5.5,3), mat="mat2", bc="bc_rect2" )
#domain1 = circle1 + rect1
#domain2 = circle2 + rect2
domain3 = (circle1 + rect1)-(circle2 + rect2) 
#domain2.Mat("mat3").Maxh(0.1) # change domain name and maxh
# add top level objects to geometry
#geo.Add(domain1)
#geo.Add(domain2)
geo.Add(domain3)
# generate mesh
mesh = geo.GenerateMesh(maxh=0.3)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'mesh_dim': 2, 'mesh_cente…

BaseWebGuiScene

- Importar la función `Draw` del módulo `ngsolve.webgui`
- Importar la clase `Mesh` y la constante `VOL` del módulo `ngsolve`
- Crear una malla utilizando la clase `Mesh` y la malla proporcionada
- Aplicar la curvatura a la malla con un grado de 3
- Definir una función característica regional (`RegionCF`) para la malla en la región de volumen (`VOL`), asignando un valor de 1 al material "mat1"
- Imprimir la función característica definida


In [3]:
from ngsolve.webgui import Draw
from ngsolve import Mesh, VOL
mesh = Mesh(mesh)
mesh.Curve(3)
cf = mesh.RegionCF(VOL, dict(mat1=1)) 
print(cf)

coef class ngfem::DomainWiseCoefficientFunction, real
  coef 1, real



- Importar la función `Integrate`, la clase `Mesh` y la constante `VOL` del módulo `ngsolve`
- Definir una nueva función característica regional (`RegionCF`) para la malla en la región de volumen (`VOL`) y asignar un valor de 1 al material "mat1"
- Calcular el área de la geometría utilizando la función `Integrate`, la función característica definida, la malla y la región de volumen, calculando regiones por separado
- Imprimir el área calculada de la geometía


In [8]:
from ngsolve import Integrate, Mesh, VOL

cf = mesh.RegionCF(VOL, dict(mat1=1))
area = Integrate(cf, mesh, VOL, region_wise=True)
print("Área de la geometría:", area)

Área de la geometría:  40.2563



In [9]:
mesh.GetBoundaries()

('bc_rect1',
 'bc_rect1',
 'bc_circle1',
 'bc_circle1',
 'bc_circle1',
 'bc_circle1',
 'bc_rect1',
 'bc_rect1',
 'bc_rect2',
 'bc_circle2',
 'bc_circle2',
 'bc_circle2',
 'bc_circle2',
 'bc_rect2')

- Definir un espacio de funciones finitas de tipo H1 en la malla, con un orden de 3 para los elementos finitos.
- Calcular y mostrar el número de grados de libertad (número de incógnitas) en este espacio de funciones finitas


In [10]:
fes = H1(mesh, order=3)
fes.ndof

4282

- Definir `u` como un objeto simbólico de función de prueba (`TrialFunction`) en el espacio de funciones finitas (`fes`).
- Definir `v` como un objeto simbólico de función de test (`TestFunction`) en el espacio de funciones finitas (`fes`).
- Crear un objeto `gfu` de tipo `GridFunction` asociado al espacio de funciones finitas (`fes`) para almacenar la solución.
- Imponer una condición de tipo Dirichlet en el valor `g` en el borde (`BND`) de la geometría.
- Dibujar la solución almacenada en `gu`.


In [13]:
u = fes.TrialFunction() 
v = fes.TestFunction()  
gfu = GridFunction(fes)  
g = 283.15 
gfu.Set(g, BND)

- Definir `u` como un objeto simbólico de función de prueba (`TrialFunction`) en el espacio de funciones finitas (`fes`).
- Definir `v` como un objeto simbólico de función de test (`TestFunction`) en el espacio de funciones finitas (`fes`).
- Crear un objeto `gfu` de tipo `GridFunction` asociado al espacio de funciones finitas (`fes`) para almacenar la solución.

- Obtener el tipo de la variable `area` y mostrarlo por pantalla.
- Definir la conductividad térmica `cond`.
- Calcular `alpha` y `beta` para la condición de Robin, donde `alpha = H` y `beta = -H*T_amb`.

- Definir la frontera `robin_boundary` donde se aplica la condición de Robin.

- Definir la forma bilineal `a` y agregar la contribución de la conductividad térmica.
- Agregar la contribución de la condición de Robin a la forma bilineal `a`.

- Ensamblar la forma bilineal `a`.

- Calcular `Q` como la potencia total dividida por el área total.

- Definir la forma lineal `f` y agregar la contribución de la fuente de calor `Q`.
- Agregar la contribución de la condición de Robin a la forma lineal `f`.

- Ensamblar a forma lineal `f`.


In [14]:
u = fes.TrialFunction()
v = fes.TestFunction()   
gfu = GridFunction(fes)   

type(area)
print(type(area))
cond = 320*100 #conductividad térmica 100*kg*cm/(K*s^3)

H = 250  # coeficiente de convección  H   W/(m^2 * K) =  (kg m^2/s^3) / (m^2 K) = (kg cm^2/s^3) / (cm^2 K)
T_amb = 283.15 
alpha = H
beta = -H*T_amb
robin_boundary = 'bc_rect1|bc_circle1|bc_circle2'

a = BilinearForm(fes)
a += grad(u)*grad(v)*cond*dx

a += alpha*u*v*ds(robin_boundary)

a.Assemble()
P=1.0E7
Q=P/area[0] 
print(Q)
f = LinearForm(fes)
f += Q*v*dx

f += -beta*v*ds(robin_boundary)

f.Assemble();

<class 'ngsolve.bla.VectorD'>
248408.2138595785


- Crear un precondicionador `c` para la forma bilineal `a` usando el método de Jacobi.
- Se ha comentado esta línea ya que no se utiliza en el código actual.

- Crear un precondicionador `c` para la forma bilineal `a` usando un solucionador directo disperso.
- Actualizar el precondicionador `c`.

- Resolver el problema de valor límite (BVP) utilizando el sistema lineal definido por la forma bilineal `a` y la forma lineal `f`, y almacenar la solución en `gfu`.
- Dibujar la solución almacenada en`gfu`.


In [15]:
c = Preconditioner(a,"direct")
c.Update()
solvers.BVP(bf=a, lf=f, gf=gfu, pre=c)
Draw(gfu)

CG iteration 1, residual = 112491.86012257091     
CG iteration 2, residual = 4.84637183305767e-09     


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

- Calcular el gradiente `grad_u` de la función de prueba `u`.
- Calcular el gradiente `grad_gfu` de la solución `gfu`.
- Dibujar el gradiente `grad_gfu` sobre la malla `mesh` con el nombre "grad_gfu.


In [16]:
grad_u = grad(u)  
grad_gfu = grad(gfu)  
Draw(grad_gfu, mesh, "grad_gfu")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene